In [13]:
from scipy.spatial.distance import cdist
from scipy.stats import mode
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from itertools import permutations, combinations

from numpy.random import default_rng
from numpy.linalg import norm
from scipy.special import softmax

In [3]:
def normalize(x):
    return (x - x.mean(axis=0))/x.std(axis=0)

In [4]:
def knn(sx, sy, tx, k=9):
    dist = cdist(tx, sx)
    sorted_idx = np.argsort(dist, axis=1)[:, :k]
    return mode(sy[sorted_idx], axis=1).mode.flatten()

In [5]:
def cbpl(sx, sy, tx, ty, th=1, alpha=0.9, num_iter=3):
    center = np.vstack([sx[sy == idx].mean(axis=0) for idx in range(65)])
    sorted_idx = None
    for _ in range(num_iter):
        dist = cdist(center, tx)
        sorted_idx = np.argsort(dist, axis=1)[:, :th]
        cand = tx[sorted_idx].mean(axis=1)
        center = alpha * center + (1 - alpha) * cand
    return tx[sorted_idx].reshape(-1, tx.shape[-1]), ty[sorted_idx]

In [6]:
def get_features(path, norm=True):
    data = np.load(path)
    ss, st = data['s'], data['t']
    ssx, ssy = ss[:, :-1], ss[:, -1]
    stx, sty = st[:, :-1], st[:, -1]
    if norm:
        return normalize(ssx), ssy, normalize(stx), sty
    return ssx, ssy, stx, sty

In [40]:
for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    t_path = f'./OfficeHome/fixbi/s{t}_t{s}.npz'
    
    ssx, ssy, stx, sty = get_features(s_path)
    ttx, tty, tsx, tsy = get_features(t_path)
    
    c1 = LinearSVC(random_state=1126, C=0.1, max_iter=5000)
    c1.fit(ssx, ssy)
    print(c1.score(stx, sty))
#     label = c1.predict(stx)
#     new_tx = np.stack([stx[label == i].mean(axis=0) for i in range(65)])
#     new_ty = np.arange(65)
    
#     old_sx = np.stack([ssx[ssy == i].mean(axis=0) for i in range(65)])
    
#     dist = cdist(old_sx, new_tx)
#     for i in range(65):
#         new_dist = cdist(old_sx[i].reshape(1, -1), ssx[ssy == i]).mean()
#         t_dist = cdist(old_sx[i].reshape(1, -1), stx[label == i]).mean()
#         print(dist[i, i], new_dist, t_dist)
#     break

KeyboardInterrupt: 

#### different space center comparison

In [54]:
for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    t_path = f'./OfficeHome/fixbi/s{t}_t{s}.npz'
    
    ssx, ssy, stx, sty = get_features(s_path)
    ttx, tty, tsx, tsy = get_features(t_path)
    
    ssc = np.stack([ssx[ssy == i].mean(axis=0) for i in range(65)])
    tsc = np.stack([tsx[tsy == i].mean(axis=0) for i in range(65)])  
    
    stc = np.stack([stx[sty == i].mean(axis=0) for i in range(65)])
    ttc = np.stack([ttx[tty == i].mean(axis=0) for i in range(65)])
    
    l = [('ssc', ssc), ('tsc', tsc), ('stc', stc), ('ttc', ttc)]
    for i in range(65):
        print(f'class{i}')
        for a, b in combinations(l, 2):
            al, av = a
            bl, bv = b
            print(f'{al}<->{bl}: {norm(av[i] - bv[i])}')
    break

class0
ssc<->tsc: 14.855554100329478
ssc<->stc: 3.9632954420995654
ssc<->ttc: 16.39119446596189
tsc<->stc: 14.247521464210147
tsc<->ttc: 5.561987437207687
stc<->ttc: 16.067070952172767
class1
ssc<->tsc: 13.635168088848191
ssc<->stc: 9.008058328828527
ssc<->ttc: 17.923844154121998
tsc<->stc: 10.283952755166142
tsc<->ttc: 9.315067445238842
stc<->ttc: 15.948941845493826
class2
ssc<->tsc: 16.940350491405884
ssc<->stc: 9.786915441873868
ssc<->ttc: 18.72616652415289
tsc<->stc: 11.91215568374334
tsc<->ttc: 8.912739531035287
stc<->ttc: 14.579175359122564
class3
ssc<->tsc: 14.999031747859293
ssc<->stc: 9.069164189420368
ssc<->ttc: 17.609531710172394
tsc<->stc: 11.575076852697743
tsc<->ttc: 7.773496779137802
stc<->ttc: 14.602760172635712
class4
ssc<->tsc: 14.902718674954121
ssc<->stc: 5.52222016146667
ssc<->ttc: 16.6611246120754
tsc<->stc: 15.07242626874351
tsc<->ttc: 4.589697449037264
stc<->ttc: 16.920174133955452
class5
ssc<->tsc: 14.159308446381324
ssc<->stc: 7.275307032002192
ssc<->ttc: 16.3

#### cbmix

In [50]:
for s, t in permutations(range(4), 2):
    s, t = 0, 3
    print(f'source {s}, target {t}')
    path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    c_path = f'./OfficeHome/cbmix_perfect/s{s}_t{t}.npz'
    
    sx, sy, _, _ = get_features(path)
    csx, csy, ctx, cty = get_features(c_path)
    
    c1 = LinearSVC(random_state=12458, C=0.1, max_iter=5000)
    c1.fit(sx, sy)

    print(c1.score(csx, csy), c1.score(ctx, cty))
    break

source 0, target 3
0.046153846153846156 0.09230769230769231


#### mixup_perfect

In [52]:
for s, t in permutations(range(4), 2):
    s, t = 1, 0
    print(f'source {s}, target {t}')
    path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    c_path = f'./OfficeHome/mixup_perfect/s{s}_t{t}.npz'
    
    sx, sy, _, _ = get_features(path)
    csx, csy, ctx, cty = get_features(c_path)
    print(csy)
    c1 = LinearSVC(random_state=12458, C=0.1, max_iter=5000)
    c1.fit(sx, sy)

    print(c1.score(csx, csy), c1.score(ctx, cty))
    break

source 0, target 1
[ 0.  0.  0. ... 64. 64. 64.]
0.9201538461538461 0.5390769230769231


### nearest neighbor for stx center

In [ ]:
# for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    t_path = f'./OfficeHome/fixbi/s{t}_t{s}.npz'
    
    ssx, ssy, stx, sty = get_features(s_path)
    ttx, tty, tsx, tsy = get_features(t_path)
    
    c1 = LinearSVC(random_state=12458, C=0.1, max_iter=5000)
    c1.fit(ssx, ssy)

    label = c1.predict(stx)
    new_tx = np.stack([stx[label == i].mean(axis=0) for i in range(65)])
    new_ty = np.arange(65)
    
    dist = cdist(new_tx, stx)
    sorted_idx = np.argsort(dist, axis=1)[:, 0]
    cand_x, ans_y = stx[sorted_idx], sty[sorted_idx]
    print(((ans_y) == new_ty).mean())
    break

#### center comparison for imagenet parameters

In [29]:
for s, t in permutations(range(4), 2):
    data_path = f'./OfficeHome/image_net/s{s}_t{t}.npz'
    
    sx, sy, tx ,ty = get_features(data_path)
    
    c1 = LinearSVC(random_state=12458, C=0.1, max_iter=5000)
    c1.fit(sx, sy)
    t_pred = c1.predict(tx)
    c2 = LinearSVC(random_state=1245, C=0.1, max_iter=5000)
    c2.fit(tx, ty)
    s_pred = c2.predict(sx)
    
    truth = np.arange(65)
    sc = np.stack([sx[sy == i].mean(axis=0) for i in range(65)])
    tpc = np.stack([tx[t_pred == i].mean(axis=0) for i in range(65)])
    spc = np.stack([sx[s_pred == i].mean(axis=0) for i in range(65)])
    tc = np.stack([tx[ty == i].mean(axis=0) for i in range(65)])
    
    l = [('sc', sc), ('tpc', tpc), ('spc', spc), ('tc', tc)]
    for i in range(65):
        print(f'class{i}')
        for a, b in combinations(l, 2):
            al, av = a
            bl, bv = b
            print(f'{al}<->{bl}: {norm(av[i] - bv[i])}')
        print('sc<->other s:', cdist(sc[i].reshape(1, -1), sx[sy == i]).mean())
        print('tc<->other t:', cdist(tc[i].reshape(1, -1), tx[ty == i]).mean())
    break

class0
sc<->tpc: 4.757830425824396
sc<->spc: 1.7763102617350952
sc<->tc: 5.305477008172781
tpc<->spc: 4.3765754870833815
tpc<->tc: 3.3319466658609613
spc<->tc: 4.8251904783628286
sc<->other s: 11.438331399215253
tc<->other t: 12.86239910230658
class1
sc<->tpc: 5.470035698332187
sc<->spc: 6.388204868882817
sc<->tc: 6.7462034225237595
tpc<->spc: 8.140879058398404
tpc<->tc: 4.4508518629451554
spc<->tc: 7.751427280930164
sc<->other s: 15.061526676342543
tc<->other t: 14.448468167263572
class2
sc<->tpc: 4.862808991685034
sc<->spc: 5.574065424093736
sc<->tc: 6.855763875146234
tpc<->spc: 4.273344513822454
tpc<->tc: 4.180755246917417
spc<->tc: 4.511858114888281
sc<->other s: 13.328952082828838
tc<->other t: 13.77267782817578
class3
sc<->tpc: 10.50676688720854
sc<->spc: 7.372514986185538
sc<->tc: 8.254403181470753
tpc<->spc: 9.503581276371838
tpc<->tc: 5.991591412642647
spc<->tc: 5.509531729285748
sc<->other s: 13.511923502025455
tc<->other t: 15.004642409088488
class4
sc<->tpc: 5.9642060026947

#### distance between stx center and ssx center

In [28]:
for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    t_path = f'./OfficeHome/fixbi/s{t}_t{s}.npz'
    
    ssx, ssy, stx, sty = get_features(s_path)
    ttx, tty, tsx, tsy = get_features(t_path)
    
    c1 = LinearSVC(random_state=1126, C=0.1, max_iter=5000)
    c1.fit(ssx, ssy)

    label = c1.predict(stx)
    new_tx = np.stack([stx[label == i].mean(axis=0) for i in range(65)])
    new_ty = np.arange(65)
    
    old_sx = np.stack([ssx[ssy == i].mean(axis=0) for i in range(65)])
    
    dist = cdist(old_sx, new_tx)
    for i in range(65):
        new_dist = cdist(old_sx[i].reshape(1, -1), ssx[ssy == i]).mean()
        t_dist = cdist(old_sx[i].reshape(1, -1), stx[label == i]).mean()
        print(dist[i, i], new_dist, t_dist)
    break

3.113970940581798 6.440459112732693 11.26728082283186
6.807086494903814 10.25645123117124 14.914544859537617
7.682448052050913 9.398734310833936 15.199483406834636
7.768203296002965 9.853032832326386 15.625999370175233
5.439224337622605 6.881466335784134 11.298813153333107
5.894114590201941 8.169746886151897 12.773819757306427
5.419875794832022 10.593849765390942 13.716910257818727
4.3535020312059975 7.963638205065823 11.508255363138336
6.401823305136886 9.067734816930841 13.925958480920059
6.497171726057615 7.986486100801861 13.360430618916034
5.299513448075538 8.722675860626984 13.797469690673555
7.627265951920709 11.5949129731141 15.136739736826112
4.503393116638213 9.776217606167535 11.898929124260853
7.276800420590684 8.957007622398038 15.532574840868568
4.632263856228464 8.012621747112405 12.407426294594734
9.688402738596068 11.20190067709059 17.07915296041812
7.768191548252729 11.169969821608843 15.969595670786815
8.677386787170269 11.403376134724274 16.63748180927508
6.93851752

In [7]:
avg = 0
for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    t_path = f'./OfficeHome/fixbi/s{t}_t{s}.npz'
    
    ssx, ssy, stx, sty = get_features(s_path)
    ttx, tty, tsx, tsy = get_features(t_path)
    
    c1 = LinearSVC(random_state=1126, C=0.1, max_iter=5000)
    c1.fit(ssx, ssy)

    s_acc = c1.score(stx, sty)
    
    c2 = LinearSVC(random_state=1126, C=0.1, max_iter=5000)
    c2.fit(tsx, tsy)
    
    t_acc = c2.score(ttx, tty)
    avg += t_acc
    print('-'*10, f'source {s}, target {t}', '-'*10)
    print(f'source acc: {s_acc}')
    print(f'target acc: {t_acc}')
print('Avg acc.:', avg/12)

---------- source 0, target 1 ----------
source acc: 0.49790697674418605
target acc: 0.73
---------- source 0, target 2 ----------
source acc: 0.66026520347508
target acc: 0.8095564700502972
---------- source 0, target 3 ----------
source acc: 0.7388164026095061
target acc: 0.8879310344827587
---------- source 1, target 0 ----------
source acc: 0.5016934801016089
target acc: 0.7455546147332769
---------- source 1, target 2 ----------
source acc: 0.6172839506172839
target acc: 0.7777777777777778
---------- source 1, target 3 ----------
source acc: 0.6328052190121156
target acc: 0.8024231127679403
---------- source 2, target 0 ----------
source acc: 0.5165114309906859
target acc: 0.8145639288738358
---------- source 2, target 1 ----------
source acc: 0.44627906976744186
target acc: 0.74
---------- source 2, target 3 ----------
source acc: 0.7220410065237651
target acc: 0.9065703634669152
---------- source 3, target 0 ----------
source acc: 0.6409822184589331
target acc: 0.893734123624047

In [14]:
th = 3
alpha = 0.1
num_iter = 1
ratio = 0.2

In [16]:
avg = 0
s_avg = 0
for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    t_path = f'./OfficeHome/fixbi/s{t}_t{s}.npz'
    ssx, ssy, stx, sty = get_features(s_path)
    ttx, tty, tsx, tsy = get_features(t_path)

    train_x, ans_y = cbpl(tsx, tsy, ttx, tty, th=th, num_iter=num_iter, alpha=alpha)
    train_y = np.tile(np.arange(65).reshape(-1, 1), th).flatten()
    print('-'*10, f'source {s}, target {t}', '-'*10)
    print('CBPL acc:', (ans_y.reshape(-1) == train_y).mean())

    c = LinearSVC(random_state=1126, C=0.1, max_iter=5000)
    c.fit(train_x, train_y)

    t_acc = c.score(ttx, tty)
    s_acc = c.score(tsx, tsy)
    avg += t_acc
    s_avg += s_acc
    print('test accuracy:', t_acc)
    print('source accuracy:', s_acc)
print('Avg acc.:', avg/12)
print('S Avg acc.:', s_avg/12)

---------- source 0, target 1 ----------
CBPL acc: 0.7794871794871795
test accuracy: 0.8234883720930233
source accuracy: 0.497883149872989
---------- source 0, target 2 ----------
CBPL acc: 0.8512820512820513
test accuracy: 0.9151806127114769
source accuracy: 0.5355630821337849
---------- source 0, target 3 ----------
CBPL acc: 0.8974358974358975
test accuracy: 0.945479962721342
source accuracy: 0.628281117696867
---------- source 1, target 0 ----------
CBPL acc: 0.7846153846153846
test accuracy: 0.8640982218458934
source accuracy: 0.48953488372093024
---------- source 1, target 2 ----------
CBPL acc: 0.7589743589743589
test accuracy: 0.8431641518061271
source accuracy: 0.44813953488372094
---------- source 1, target 3 ----------
CBPL acc: 0.8666666666666667
test accuracy: 0.9212488350419384
source accuracy: 0.5251162790697674
---------- source 2, target 0 ----------
CBPL acc: 0.841025641025641
test accuracy: 0.844623200677392
source accuracy: 0.6540923639689071
---------- source 2, ta

In [39]:
for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
  
    ssx, ssy, stx, sty = get_features(s_path)

    ssc = np.stack([ssx[ssy == i].mean(axis=0) for i in range(65)])    
    stc = np.stack([stx[sty == i].mean(axis=0) for i in range(65)])
    
    c1 = LinearSVC(random_state=13743, C=0.1, max_iter=5000)
    c1.fit(ssx, ssy)
    
    st_pred = c1.predict(stx)
    
    for i in range(65):
        print(f'class {i}')
        # pred 0 data in target
        same_mean = stx[(st_pred == i) & (sty == i)].mean(axis=0)
        # True 0 data in target
        diff_mean = stx[(st_pred != i) & (sty == i)].mean(axis=0)

        pred_mean = stx[st_pred == i].mean(axis=0)
        s_dist = cdist(ssc[i].reshape(1, -1), ssx[ssy == i]).mean()
        
#         print(f'Correct mean <-> target center: {norm(same_mean-stc[i])}')
#         print(f'Incorrect mean <-> target center: {norm(diff_mean-stc[i])}')
        print(f'Source center <-> target center: {norm(ssc[i] - stc[i])}')
        print(f'Source center <-> pred target center: {norm(pred_mean - ssc[i])}')
        print(f'Predict target center <-> target center: {norm(pred_mean - stc[i])}')
        print(f'Source avg dist: {s_dist}')
#         print(f'Accuracy: {c1.score(stx[sty == i], i*np.ones((sty == i).sum()))}')
    print(ssy.shape, sty.shape)
    break

class 0
Source center <-> target center: 3.9632954420995654
Source center <-> pred target center: 3.113970940581798
Predict target center <-> target center: 2.4427068464656663
Source avg dist: 6.440459112732693
class 1
Source center <-> target center: 9.008058328828527
Source center <-> pred target center: 6.807086494903814
Predict target center <-> target center: 4.331181269996933
Source avg dist: 10.25645123117124
class 2
Source center <-> target center: 9.786915441873868
Source center <-> pred target center: 7.682448052050914
Predict target center <-> target center: 4.401446408915221
Source avg dist: 9.398734310833936
class 3
Source center <-> target center: 9.069164189420368
Source center <-> pred target center: 7.768203296002964
Predict target center <-> target center: 3.802359552404894
Source avg dist: 9.853032832326386
class 4
Source center <-> target center: 5.52222016146667
Source center <-> pred target center: 5.439224337622605
Predict target center <-> target center: 1.82470

#### See varaince

In [38]:
for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
  
    ssx, ssy, stx, sty = get_features(s_path, norm=False)

    ssc = np.stack([ssx[ssy == i].mean(axis=0) for i in range(65)])    
    stc = np.stack([stx[sty == i].mean(axis=0) for i in range(65)])
    
    c1 = LinearSVC(random_state=13743, C=0.1, max_iter=5000)
    c1.fit(ssx, ssy)
    
    st_pred = c1.predict(stx)
    
    for i in range(65):
        print(f'class {i}')
        true_tx = ssx[ssy == i]
        norm_true_tx = (true_tx - true_tx.mean(axis=0))/true_tx.std(axis=0)
        
        print(norm_true_tx)
        break
#         pred_tx = stx[st_pred == i]
#         norm_pred_tx = (pred_tx - pred_tx.mean(axis=0))/pred_tx.std(axis=0)
#         print(norm(norm_pred_tx - norm_true_tx))
    break

class 0
[[ 0.40586963 -0.88506537 -0.09613184 ... -0.18148455 -0.50622758
   1.21414509]
 [ 0.1056807   1.15089375  2.13831555 ... -0.23118617 -1.9497277
   0.45642478]
 [-1.21322456  0.23124784 -0.28578164 ...  0.12921967  0.15329232
  -0.13920284]
 ...
 [ 0.17928674  0.69416666 -1.87808682 ...  0.83323987  0.42359814
  -1.34151892]
 [-1.26699791 -2.89743137  0.72685268 ...  0.12858556  0.28372056
   0.72891995]
 [-0.4297831  -1.51424923  0.03743405 ... -0.50456116 -0.52150017
   0.91585646]]


#### with / without normalization

In [22]:
for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    sx, sy, tx, ty = get_features(s_path, norm=False)
    nsx, nsy, ntx, nty = get_features(s_path)

    sc = np.stack([sx[sy == i].mean(axis=0) for i in range(65)])    
    tc = np.stack([tx[ty == i].mean(axis=0) for i in range(65)])
    
    c1 = LinearSVC(random_state=13743, C=0.1, max_iter=5000)
    c1.fit(sx, sy)
    
    nsc = np.stack([nsx[nsy == i].mean(axis=0) for i in range(65)])    
    ntc = np.stack([ntx[nty == i].mean(axis=0) for i in range(65)])
    
    c2 = LinearSVC(random_state=13743, C=0.1, max_iter=5000)
    c2.fit(nsx, nsy)
    
    labels = np.arange(65)
    print(c1.score(tc, labels), c2.score(ntc, labels))

0.8769230769230769 0.9076923076923077
0.9384615384615385 0.9384615384615385
0.9692307692307692 0.9692307692307692
0.9076923076923077 0.9384615384615385
0.8769230769230769 0.9230769230769231
0.9846153846153847 0.9846153846153847
0.9230769230769231 0.9538461538461539
0.8769230769230769 0.9076923076923077
0.9692307692307692 0.9692307692307692
0.9846153846153847 1.0
0.8307692307692308 0.9384615384615385
1.0 1.0


In [16]:
def prototype_classifier(X, C, th=2):
    dist = cdist(C, X)
    prob = softmax(-dist, axis=0)
    return prob.argsort(axis=0)[-th:, :].astype(float), prob

#### Prototypical Classifier with multiple pseudo center

In [21]:
avg = 0
label_dict = {i: (a, b) for i, (a, b) in enumerate(combinations(range(65), 2))}
for s, t in permutations(range(4), 2):  
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    ssx, ssy, stx, sty = get_features(s_path, norm=False)
    sty = sty.astype(float)
    ssc = np.stack([ssx[ssy == i].mean(axis = 0) for i in range(65)])
    
    pred, prob = prototype_classifier(stx, ssc, th=2)
    p = np.array([True if y in pr else False for y, pr in zip(sty, pred.T)], dtype=bool)
    score = p.mean()
#     conf = prob.max(axis=0)
#     cri = conf > 0.8
#     good_pred, good_y = pred[cri], sty[cri]
#     remain_x, remain_y = stx[~cri], sty[~cri]
    
#     remain_pred, remain_prob = prototype_classifier(remain_x, psc)
#     remain_pred = [label_dict[int(i)] for i in remain_pred]
#     cor_pred = []
#     p, tot = 0, 0
#     for (a, b), x, y in zip(remain_pred, remain_x, remain_y):
#         a_norm = norm(ssc[int(a)] - x)
#         b_norm = norm(ssc[int(b)] - x)
#         cor_pred.append(a if a_norm < b_norm else b)
#         if y in [a, b]:
#             tot += 1
#             if y == cor_pred[-1]:
#                 p += 1
#     cor_pred = np.array(cor_pred, dtype=float)
    
#     all_pred = np.r_[good_pred, cor_pred]
#     all_y = np.r_[good_y, remain_y]
#     cor_score = p/tot
#     score = (all_pred == all_y).mean()
    
#     pred = [label_dict[int(i)] for i in pred]
    
#     cor_pred = []
#     p, tot = 0, 0
#     for (a, b), x, y in zip(pred, stx, sty):
        
#         a_norm = norm(ssc[int(a)] - x)
#         b_norm = norm(ssc[int(b)] - x)
#         cor_pred.append(a if a_norm < b_norm else b)
#         if a != b:
#             tot+=1
#             if y == cor_pred[-1]:
#                 p += 1
#     cor_pred = np.array(cor_pred, dtype=float)
#     correct = np.array([True if y in p else False for y, p in zip(sty, pred)])
#     score = correct.mean()
#     pred = prototype_classifier(stx, ssc)
#     print('-'*10, f'source {s}, target {t}', '-'*10)
#     print(tot / len(remain_y))
#     print(cor_score)
    print(score)
    avg += score
print('Avg acc.:', avg/12)

0.6402325581395348
0.7812071330589849
0.853448275862069
0.6608806096528366
0.7517146776406035
0.7646784715750233
0.6710414902624894
0.5674418604651162
0.8511183597390494
0.7633361558001693
0.623953488372093
0.881572930955647
Avg acc.: 0.7342188342936348


#### svm for source @ source space

In [23]:
avg = 0
for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    ssx, ssy, stx, sty = get_features(s_path, norm=False)
    
    c1 = LinearSVC(random_state=1128, C=0.1, max_iter=5000)
    c1.fit(ssx, ssy)
    pred = c1.predict(stx)
    score = (pred == sty).mean()
    print('-'*10, f'source {s}, target {t}', '-'*10)
    print(score)
    avg += score
print('Avg acc.:', avg/12)

---------- source 0, target 1 ----------
0.4890697674418605
---------- source 0, target 2 ----------
0.6566072245084591
---------- source 0, target 3 ----------
0.7404473438956197
---------- source 1, target 0 ----------
0.4932260795935648
---------- source 1, target 2 ----------
0.6095107453132145
---------- source 1, target 3 ----------
0.6255824790307549
---------- source 2, target 0 ----------
0.5080440304826418
---------- source 2, target 1 ----------
0.4323255813953488
---------- source 2, target 3 ----------
0.7187791239515378
---------- source 3, target 0 ----------
0.6401354784081287
---------- source 3, target 1 ----------
0.49511627906976746
---------- source 3, target 2 ----------
0.7821216278006401
Avg acc.: 0.5992471467409616


#### svm for source center @ source space

In [22]:
avg = 0
for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    ssx, ssy, stx, sty = get_features(s_path, norm=False)
    
    ssc = np.stack([ssx[ssy == i].mean(axis = 0) for i in range(65)])
    c1 = LinearSVC(random_state=1128, C=0.1, max_iter=5000)
    labels = np.arange(65)
    c1.fit(ssc, labels)
    pred = c1.predict(stx)
    score = (pred == sty).mean()
    print('-'*10, f'source {s}, target {t}', '-'*10)
    print(score)
    avg += score
print('Avg acc.:', avg/12)

---------- source 0, target 1 ----------
0.493953488372093
---------- source 0, target 2 ----------
0.6611796982167353
---------- source 0, target 3 ----------
0.7409133271202236
---------- source 1, target 0 ----------
0.5177815410668924
---------- source 1, target 2 ----------
0.6127114769090077
---------- source 1, target 3 ----------
0.6293103448275862
---------- source 2, target 0 ----------
0.5139712108382727
---------- source 2, target 1 ----------
0.4416279069767442
---------- source 2, target 3 ----------
0.7327586206896551
---------- source 3, target 0 ----------
0.6418289585097375
---------- source 3, target 1 ----------
0.4958139534883721
---------- source 3, target 2 ----------
0.7841792409693644
Avg acc.: 0.6055024806653903


#### Prototypical Classifier @ Source Space

In [49]:
avg = 0
for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    ssx, ssy, stx, sty = get_features(s_path, norm=False)
    
    ssc = np.stack([ssx[ssy == i].mean(axis = 0) for i in range(65)])
    pred, prob = prototype_classifier(stx, ssc)
    sty = sty.astype(float)
    score = (pred == sty).mean()
    print('-'*10, f'source {s}, target {t}', '-'*10)
    print(score)
    avg += score
print('Avg acc.:', avg/12)

---------- source 0, target 1 ----------
0.5130232558139535
---------- source 0, target 2 ----------
0.6801554641060814
---------- source 0, target 3 ----------
0.7562907735321528
---------- source 1, target 0 ----------
0.5381033022861982
---------- source 1, target 2 ----------
0.6332876085962506
---------- source 1, target 3 ----------
0.6498136067101584
---------- source 2, target 0 ----------
0.5537679932260796
---------- source 2, target 1 ----------
0.46069767441860465
---------- source 2, target 3 ----------
0.7467381174277726
---------- source 3, target 0 ----------
0.6583403895004234
---------- source 3, target 1 ----------
0.5076744186046511
---------- source 3, target 2 ----------
0.7960676726108825
Avg acc.: 0.6244966897361007


#### Prototypical Network Classifier

In [64]:
avg = 0
for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    t_path = f'./OfficeHome/fixbi/s{t}_t{s}.npz'
    ssx, ssy, stx, sty = get_features(s_path, norm=False)
    ttx, tty, tsx, tsy = get_features(t_path, norm=False)
    
    tsc = np.stack([tsx[tsy == i].mean(axis=0) for i in range(65)])
    ttc = np.stack([ttx[tty == i].mean(axis=0) for i in range(65)])
    labels = np.arange(65)
    print('-'*10, f'source {s}, target {t}', '-'*10)
    pred = prototype_classifier(ttx, tsc)
    tty = tty.astype(float)
    score = (pred == tty).mean()
#     print('-'*10, f'source {s}, target {t}', '-'*10)
#     score = c1.score(ttx, tty)
    print(score)
    avg += score
print('Avg acc.:', avg/12)

---------- source 0, target 1 ----------
0.8702325581395349
---------- source 0, target 2 ----------
0.9364426154549611
---------- source 0, target 3 ----------
0.9773998136067101
---------- source 1, target 0 ----------
0.9513124470787468
---------- source 1, target 2 ----------
0.9119798811156836
---------- source 1, target 3 ----------
0.9305684995340168
---------- source 2, target 0 ----------
0.9149026248941575
---------- source 2, target 1 ----------
0.8930232558139535
---------- source 2, target 3 ----------
0.9713420316868593
---------- source 3, target 0 ----------
0.9771380186282811
---------- source 3, target 1 ----------
0.9251162790697675
---------- source 3, target 2 ----------
0.9830818472793782
Avg acc.: 0.9368783226918375


#### without pseudo labeling on target space

In [31]:
avg = 0
for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    t_path = f'./OfficeHome/fixbi/s{t}_t{s}.npz'
    ssx, ssy, stx, sty = get_features(s_path, norm=False)
    ttx, tty, tsx, tsy = get_features(t_path, norm=False)
    
    tsc = np.stack([tsx[tsy == i].mean(axis=0) for i in range(65)])
    ttc = np.stack([ttx[tty == i].mean(axis=0) for i in range(65)])
    labels = np.arange(65)
    
    c1 = LinearSVC(random_state=14582, C=0.1, max_iter=5000)
    c1.fit(tsc, labels)
    print('-'*10, f'source {s}, target {t}', '-'*10)
    score = c1.score(ttx, tty)
    print(score)
    avg += score
print('Avg acc.:', avg/12)

---------- source 0, target 1 ----------
0.8837209302325582
---------- source 0, target 2 ----------
0.9467306812985825
---------- source 0, target 3 ----------
0.9760018639328985
---------- source 1, target 0 ----------
0.9606265876375952
---------- source 1, target 2 ----------
0.9213534522176497
---------- source 1, target 3 ----------
0.9541006523765144
---------- source 2, target 0 ----------
0.9314140558848434
---------- source 2, target 1 ----------
0.8958139534883721
---------- source 2, target 3 ----------
0.9734389561975769
---------- source 3, target 0 ----------
0.9767146486028789
---------- source 3, target 1 ----------
0.9290697674418604
---------- source 3, target 2 ----------
0.9803383630544125
Avg acc.: 0.9441103260304785


In [33]:
avg = 0
for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
    t_path = f'./OfficeHome/fixbi/s{t}_t{s}.npz'
    ssx, ssy, stx, sty = get_features(s_path, norm=False)
    ttx, tty, tsx, tsy = get_features(t_path, norm=False)
    
    c1 = LinearSVC(random_state=13241, C=0.1, max_iter=5000)
    c1.fit(tsx, tsy)
    print('-'*10, f'source {s}, target {t}', '-'*10)
    score = c1.score(ttx, tty)
    print(score)
    avg += score
print('Avg acc.:', avg/12)

---------- source 0, target 1 ----------
0.7011627906976744
---------- source 0, target 2 ----------
0.793095564700503
---------- source 0, target 3 ----------
0.8837371854613234
---------- source 1, target 0 ----------
0.7341236240474175
---------- source 1, target 2 ----------
0.7629172382258802
---------- source 1, target 3 ----------
0.7893755824790307
---------- source 2, target 0 ----------
0.7904318374259103
---------- source 2, target 1 ----------
0.7295348837209302
---------- source 2, target 3 ----------
0.8986486486486487
---------- source 3, target 0 ----------
0.8852667231160034
---------- source 3, target 1 ----------
0.7788372093023256
---------- source 3, target 2 ----------
0.9167809785093736
Avg acc.: 0.8053260221945852


#### Hyper parameter selection

In [14]:
# rng = np.random.default_rng(12345)
c_list = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10]
total_avg = []
for c in c_list:
    avg = 0
    for s, t in permutations(range(4), 2):
        s_path = f'./OfficeHome/fixbi/s{s}_t{t}.npz'
        t_path = f'./OfficeHome/fixbi/s{t}_t{s}.npz'
        ssx, ssy, stx, sty = get_features(s_path, norm=False)
        ttx, tty, tsx, tsy = get_features(t_path, norm=False)

        tsc = np.stack([tsx[tsy == i].mean(axis=0) for i in range(65)])
        ttc = np.stack([ttx[tty == i].mean(axis=0) for i in range(65)])
        labels = np.arange(65)

        c1 = LinearSVC(random_state=12453, C=c, max_iter=5000)
#         c1.fit(tsc, labels)
        c1.fit(tsx, tsy)
#         print('-'*10, f'source {s}, target {t}', '-'*10)
        score = c1.score(ttx, tty)
#         print(score)
        avg += score
    print(f'C = {c}, Avg acc.:', avg/12)
    total_avg.append(avg/12)
print('Total Avg acc.:', np.mean(total_avg))
print('std:', np.std(total_avg))

C = 0.0001, Avg acc.: 0.8634686205510889
C = 0.001, Avg acc.: 0.8975306780887019
C = 0.01, Avg acc.: 0.8759543175594581
C = 0.1, Avg acc.: 0.8053260221945852


/opt/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("L

C = 1, Avg acc.: 0.7451494320231244


/opt/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("L

C = 10, Avg acc.: 0.7139829738912523
Total Avg acc.: 0.8169020073847019
std: 0.06835126223855716


/opt/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
